# Deep learning для классификации картинок

Семинар состоит из 2 частей

1. Ознакомьтесь со структурой обычного обучающего скрипта и потренируйте старую добрую сеть, подобную vgg
2. Улучшите качество с помощью сети, подобной resnet

Но сначала посмотрим на данные

<img src="./vgg-neural-network-architecture.png" alt="Drawing" style="width:90%"/> </td>

# Tiny ImageNet dataset
На этом семинаре мы сосредоточимся на задаче распознавания изображений на Tiny Image Net dataset. Этот набор данных содержит
* 100 тысяч изображений размером 3x64x64
* 200 различных классов: змеи, пауки, кошки, грузовики, кузнечики, чайки и т.д.

На самом деле, это подмножество набора данных ImageNet с изображениями, уменьшенными в 4 раза.

## Image examples



<tr>
    <td> <img src="https://github.com/yandexdataschool/Practical_DL/blob/sem3spring2019/week03_convnets/tinyim3.png?raw=1" alt="Drawing" style="width:90%"/> </td>
    <td> <img src="https://github.com/yandexdataschool/Practical_DL/blob/sem3spring2019/week03_convnets/tinyim2.png?raw=1" alt="Drawing" style="width:90%"/> </td>
</tr>


<tr>
    <td> <img src="https://github.com/yandexdataschool/Practical_DL/blob/sem3spring2019/week03_convnets/tiniim.png?raw=1" alt="Drawing" style="width:90%"/> </td>
</tr>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Step 0 - data loading

In [ ]:
from tiny_img import download_tinyImg200
data_path = '.'
download_tinyImg200(data_path)

Выходные данные были обрезаны до нескольких последних строк (5000).
207167488 / 248100043
207175680 / 248100043
207183872 / 248100043
207192064 / 248100043
207200256 / 248100043
207208448 / 248100043
207216640 / 248100043
207224832 / 248100043
207233024 / 248100043
207241216 / 248100043
207249408 / 248100043
207257600 / 248100043
207265792 / 248100043
207273984 / 248100043
207282176 / 248100043
207290368 / 248100043
207298560 / 248100043
207306752 / 248100043
207314944 / 248100043
207323136 / 248100043
207331328 / 248100043
207339520 / 248100043
207347712 / 248100043
207355904 / 248100043
207364096 / 248100043
207372288 / 248100043
207380480 / 248100043
207388672 / 248100043
207396864 / 248100043
207405056 / 248100043
207413248 / 248100043
207421440 / 248100043
207429632 / 248100043
207437824 / 248100043
207446016 / 248100043
207454208 / 248100043
207462400 / 248100043
207470592 / 248100043
207478784 / 248100043
207486976 / 248100043
207495168 / 248100043
207503360 / 248100043
20751155

## Part 1. Training script structure and vgg-like network

Чтобы обучить нейронную сеть, надо решить 5 задач:
1. data loader (data provider) - как загружать и дополнять данные для обучения nn
2. neural network architecture - что будет обучаться
3. loss function (+ auxilary metrics on train and validation set) - как проверить качество нейронной сети
4. optiimzer and training schedule - как будет обучаться нейронная сеть
5. "Train loop" - что именно нужно делать для каждого пакета, как часто проверять ошибку проверки, как часто сохранять сеть и т.д. Этот код можно было бы написать в общем виде и повторно использовать в разных сценариях обучения.

In [ ]:
import torch
import torchvision
from torchvision import transforms
import tqdm

def get_computing_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

device = get_computing_device()
print(f"Our main computing device is '{device}'")

Our main computing device is 'cuda:0'


### 1.1 Data loader and data augmentation
Обычно для манипулирования данными используются две связанные абстракции:
- Dataset (`torch.utils.data.Dataset` и его подклассы из `torchvision.datasets") - некоторый черный ящик, который хранит и предварительно обрабатывает отдельные элементы dataset. В частности, на этом уровне обычно находятся дополнения для отдельных образцов.
- DataLoader (`torch.utils.data.DataLoader`) - структура, объединяющая отдельные элементы в пакетном режиме.

Давайте разберемся с обучающим набором данных. Вот несколько простых дополнений, которые мы собираемся использовать в наших экспериментах:

In [ ]:
train_trainsforms = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.RandomRotation(5),
        # YOUR CODE : examine torchvision.transforms package, find transformation for color jittering
        # and add it with proper parameters.
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        # you may add any other transforms here
    ]
)

Для набора обучающих данных мы будем использовать пользовательский набор данных, который будет хранить все обучающие данные в оперативной памяти. Если у вас недостаточно оперативной памяти, вы можете использовать `torch vision.datasets.ImageFolder()`.

In [ ]:
import tiny_img_dataset

# you may use torchvision.datasets.ImageFolder() with the same parameters for loading train dataset
train_dataset = tiny_img_dataset.TinyImagenetRAM(
    "tiny-imagenet-200/train", transform=train_trainsforms
)

tiny-imagenet-200/train: 100%|██████████| 200/200 [01:21<00:00,  2.44it/s]


Теперь проверка. Взгляните на папку `tiny-imagenet-200/val` и сравните ее с папкой `tiny-imagenet-200/train`. Выглядит по-другому, не так ли? Таким образом, мы не можем использовать `TinyImagenetRAM` для загрузки набора данных для проверки. Давайте вместо этого напишем пользовательский набор данных, но с таким же поведением, как у `TinyImagenetRAM`.

In [ ]:
from torch.utils.data import Dataset
import os
from PIL import Image

import numpy as np


class TinyImagenetValDataset(Dataset):
    def __init__(self, root, transform=transforms.ToTensor()):
        super().__init__()

        self.root = root
        with open(os.path.join(root, "val_annotations.txt")) as f:
            annotations = []
            for line in f:
                img_name, class_label = line.split("\t")[:2]
                annotations.append((img_name, class_label))

        # 1. define self.classes - list of sorted class labels from annotations
        # it should look like self.classes from "TinyImagenetRAM"
        # YOUR CODE
        self.classes = sorted(set(class_label for _, class_label in annotations))

        assert len(self.classes) == 200, len(self.classes)
        assert all(
            self.classes[i] < self.classes[i + 1] for i in range(len(self.classes) - 1)
        ), "classes should be ordered"
        assert all(
            isinstance(elem, type(annotations[0][1])) for elem in self.classes
        ), "your just need to reuse class_labels"

        # 2. self.class_to_idx - dict from class label to class index
        self.class_to_idx = {item: index for index, item in enumerate(self.classes)}

        self.transform = transform

        self.images, self.targets = [], []
        for img_name, class_name in tqdm.tqdm(annotations, desc=root):
            img_path = os.path.join(root, "images", img_name)
            # 3. load image and store it in self.images (your may want to use tiny_img_dataset.read_rgb_image)
            # store the class index in self.targets
            # YOUR CODE
            image = tiny_img_dataset.read_rgb_image(img_path)

            # if img.mode != 'RGB':
            #     img = img.convert('RGB')
            # if img.size != (64, 64):
            #     img = img.resize((64, 64))

            assert image.shape == (64, 64, 3), image.shape
            self.images.append(Image.fromarray(image))
            self.targets.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        # take image and its target label from "self.images" and "self.targets",
        # transform the image using self.transform and return the transformed image and its target label

        # YOUR CODE
        image = self.images[index]
        image = self.transform(image)
        target = self.targets[index]

        return image, target

Давайте, наконец, загрузим набор данных для проверки. Обычно вам не применять аугментации для проверки.

In [ ]:
val_dataset = TinyImagenetValDataset(
    "tiny-imagenet-200/val", transform=transforms.ToTensor()
)

assert all(
    train_dataset.classes[i] == val_dataset.classes[i] for i in range(200)
), "class order in train and val datasets should be the same"
assert all(
    train_dataset.class_to_idx[elem] == val_dataset.class_to_idx[elem]
    for elem in train_dataset.classes
), "class indices should be the same"

tiny-imagenet-200/val: 100%|██████████| 10000/10000 [00:08<00:00, 1183.97it/s]


Для большинства случаев будет достаточно `DataLoader` по умолчанию.

In [ ]:
batch_size = 64
train_batch_gen = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
)

In [ ]:
val_batch_gen = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=2
)

### 1.2 Определение нейронной сети

"Сеть, подобная VGG", обычно означает, что сеть представляет собой последовательность сверток с MaxPooling для понижающей размерности.

Вот таблица из оригинальной статьи ["Very Deep Convolutional Networks for Large-Scale Image Recognition"].(https://arxiv.org/abs/1409.1556), который описывает классические конфигурации сетей VGG (часто называемые VGG-A, VGG-B и т.д. С использованием имени столбца в качестве идентификатора или VGG16, VGG19 и т.д. с использованием количества слоев в качестве идентификатора).

![image.png](https://pytorch.org/assets/images/vgg.png)

Эти сетевые конфигурации были разработаны для набора данных ImageNet. Поскольку изображения в tiny-imagenet имеют пониженный размер в 4 раза, мы собираемся разработать нашу собственную конфигурацию, уменьшив:
1) количество слоев;
2) количество нейронов в слоях;
3) количество слоев с максимальным объединением, которые уменьшают выборку карт объектов

Конфигурация нашей сети будет выглядеть следующим образом [Conv(16), Conv(16), MaxPool] + [Conv(32), Conv(32), MaxPool] + [Conv(64), Conv(64), MaxPool] + [Conv(128), Conv(128)] + [GlobalAveragePooling] + [FC(200) + softmax]


Мы используем Conv(128) и GlobalAveragePooling вместо image flattening и слоев FC для уменьшения количества параметров.

In [ ]:
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

И еще кое-что. VGG был разработан до того, как был придуман BatchNormalization. В настоящее время было бы глупо, если бы мы не использовали BatchNormalization в нашей сети. Итак, давайте определим простой модуль, содержащий свертку, BatchNormalization и relu, и построим нашу сеть, используя этот модуль. Вот также реализация GlobalAveragePooling, приведенная для вас в качестве примера пользовательского модуля.

In [ ]:
#!L
class GlobalAveragePool(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        return torch.mean(x, dim=self.dim)


class ConvBNRelu(nn.Module):
    def __init__(
        self, in_channels, out_channels, kernel_size, stride=1, padding="same"
    ):
        super().__init__()

        # Вычисление отступов для сохранения размерности
        if padding == "same":
            padding = (kernel_size - 1) // 2
        else:
            padding = 0

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


def create_vgg_like_network(config=None):
    """
    Creates VGG like network according to config
    """
    model = nn.Sequential()

    default_config = [[16, 16], [32, 32], [64, 64], [128, 128]]
    config = config or default_config

    in_channels = 3
    layer_counter = 0  # Счетчик для генерации уникальных имен

    for block_index in range(len(config)):
        for layer_index_in_block in range(len(config[block_index])):
            out_channels = config[block_index][layer_index_in_block]

            layer_name = f"conv_block{block_index}_layer{layer_counter}"
            model.add_module(
                layer_name, ConvBNRelu(in_channels, out_channels, kernel_size=3)
            )

            in_channels = out_channels
            layer_counter += 1

        if block_index != len(config) - 1:
            pool_name = f"mp_{block_index}"
            model.add_module(pool_name, nn.MaxPool2d(3, stride=2))

    model.add_module("pool", GlobalAveragePool(dim=(2, 3)))
    model.add_module("logits", nn.Linear(out_channels, 200))
    return model

Вот и создана наша модель!

In [ ]:
model = create_vgg_like_network()
model = model.to(device)

### 1.3 Определение функции потерь

Обычно в качестве функции потерь для классификации изображений используется cross-entropy (отрицательное логарифмическое правдоподобие).

In [ ]:
def compute_loss(predictions, gt):
    return F.cross_entropy(predictions, gt).mean()

### 1.4 Optimizer and training schedule

Давайте обучим нашу сеть, используя Adam с параметрами по умолчанию.

Для обучения с помощью `torch.optim.SGD` вам обычно нужно определить training schedule - способ, как снизить learning rate во время тренировки. Но поскольку в adam все градиенты масштабируются по моменту, эффект от правильного графика тренировок не так важен для обучения, как в SGD. Поэтому мы будем действовать как ленивые специалисты по обработке данных и не будем использовать шедулер вообще. Но вы можете поиграть с шедулером, используя, например, `torch.optim.lr_scheduler.ExponentialLR`, смотрите [документацию](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate) с объяснением, как это использовать.

In [ ]:
opt = torch.optim.Adam(model.parameters())

### 1.5 Цикл обучения

Давайте объединим ранее определенные элементы вместе.

In [ ]:
import numpy as np
import time
import torch.nn.functional as F  # Добавлен импорт для функции потерь

def eval_model(model, data_generator):
    accuracy = []
    model.train(False) # disable dropout / use averages for batch_norm
    with torch.no_grad():
        for X_batch, y_batch in data_generator:
            X_batch = X_batch.to(device)
            logits = model(X_batch)
            y_pred = logits.max(1)[1].data
            accuracy.append(np.mean((y_batch.cpu() == y_pred.cpu()).numpy()))
    return np.mean(accuracy)


def train_model(model, optimizer, train_data_generator):
    train_loss = []
    model.train(True) # enable dropout / batch_norm training behavior
    for (X_batch, y_batch) in tqdm.tqdm(train_data_generator):
        opt.zero_grad()

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        # Вычисляем выходы модели
        predictions = model(X_batch)

        # Вычисляем потерю (кросс-энтропия)
        loss = F.cross_entropy(predictions, y_batch)

        # backward
        loss.backward()
        optimizer.step()

        # metrics
        train_loss.append(loss.cpu().data.numpy())
    return np.mean(train_loss)


def train_loop(model, optimizer, train_data_generator, val_data_generator, num_epochs):
    """
    num_epochs - total amount of full passes over training data
    """
    for epoch in range(num_epochs):
        start_time = time.time()

        train_loss = train_model(model, optimizer, train_data_generator)

        val_accuracy = eval_model(model, val_data_generator)

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss (in-iteration): \t{:.6f}".format(train_loss))
        print("  validation accuracy: \t\t\t{:.2f} %".format(val_accuracy * 100))

### 1.6 Обучение

Вся подготовка завершена, пора запускать обучение!

Обычно после обучения в течение 30 эпох вы должны получить нейронную сеть, которая предсказывает метки с точностью более 40%.

In [ ]:
train_loop(model, opt, train_batch_gen, val_batch_gen, num_epochs=5)

100%|██████████| 1563/1563 [03:57<00:00,  6.57it/s]


Epoch 1 of 5 took 239.551s
  training loss (in-iteration): 	4.394727
  validation accuracy: 			12.06 %


100%|██████████| 1563/1563 [03:49<00:00,  6.82it/s]


Epoch 2 of 5 took 231.016s
  training loss (in-iteration): 	3.731887
  validation accuracy: 			18.89 %


100%|██████████| 1563/1563 [03:55<00:00,  6.64it/s]


Epoch 3 of 5 took 237.471s
  training loss (in-iteration): 	3.395469
  validation accuracy: 			23.71 %


100%|██████████| 1563/1563 [03:55<00:00,  6.65it/s]


Epoch 4 of 5 took 236.927s
  training loss (in-iteration): 	3.192448
  validation accuracy: 			26.07 %


100%|██████████| 1563/1563 [03:52<00:00,  6.71it/s]


Epoch 5 of 5 took 234.569s
  training loss (in-iteration): 	3.040678
  validation accuracy: 			29.26 %


## Part 2. Say Hello to ResNets

В этой части вам нужно переопределить вашу модель, все остальное останется прежним. Как и в случае с VGG, мы собираемся определить модель, подобную ResNet, а не классическую архитектуру, разработанную для классификации ImageNet.

"ResNet-подобный" обычно означает, что ваша сеть состоит из "residual блоков". Существует два широко используемых типа блоков: с двумя и с тремя свертками:
![resnet_blocks](https://miro.medium.com/max/613/1*zS2ChIMwAqC5DQbL5yD9iQ.png)

На практике часто используются блоки с тремя свертками, поскольку они позволяют построить более глубокую сеть с меньшими параметрами. Блоки с двумя свертками обычно используются для сравнения с non-residual сетями, особенно с VGG и AlexNet.

Вот таблица из статьи "[Deep Residual Learning for Image Recognition]"(https://arxiv.org/pdf/1512.03385.pdf), в которой описываются классические конфигурации сетей ResNet. Обычно их называют ResNet-18, ResNet-34 и так далее, используя количество слоев в качестве идентификатора. Обратите внимание, что сети, начиная с ResNet-50, основаны на 3-сверточных блоках. На самом деле ResNet-18 и ResNet-34 были представлены только для сравнения с VGG, в то время как ResNet-50 обычно используется на практике в качестве хорошего бейслайна.

![изображение](https://miro.medium.com/max/2400/1*aq0q7gCvuNUqnMHh4cpnIw.png)

Как и в случае с VGG, мы собираемся создать нашу собственную конфигурацию для сети. Давайте используем 2-сверточных блока для сравнения с vgg и возьмем сеть типа [Conv7x7 - 32] + [conv32-block, conv32-block] + [conv64-block, conv64-block] + [conv128-block, conv128-block] + [GlobalAveragePooling] + fc200 + softmax

По сравнению с ResNet18, мы уменьшили количество фильтров и убрали max-pooling в начале и в последнем наборе сверток

In [ ]:
class ResNetBlock2(nn.Module):
    """
    Module implements the following function:

    output = relu(F(input) + Residual(input)), where:
        Residual(x) = Conv + bn + relu + conv + bn
        F(x) = x                                        , if in_channels == out_channels and stride == 1
             = Conv1x1(in_channel, out_channel, stride) , otherwise
    """

    def __init__(
        self, in_channels, out_channels, kernel_size=3, stride=1, padding="same"
    ):
        super().__init__()

        # Вычисление отступов
        if padding == "same":
            padding = (kernel_size - 1) // 2
        else:
            padding = 0

        # Residual branch (две последовательные свертки)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, 1, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu2 = nn.ReLU()

        self.conv3 = None  # conv for main branch adopatation
        if in_channels != out_channels or stride != 1:
            self.conv3 = nn.Conv2d(in_channels, out_channels, 1, stride, padding=0)

    def forward(self, x):
        # YOUR CODE: compute residual branch,
        # DON'T OVERRIDE 'x' as you will need it

        # Residual branch
        residual = self.conv1(x)
        residual = self.bn1(residual)
        residual = self.relu1(residual)

        residual = self.conv2(residual)
        residual = self.bn2(residual)

        if self.conv3 is not None:
            x = self.conv3(x)

        result = self.relu2(residual + x)
        return result


def create_resnet_like_network():
    model = nn.Sequential()

    config = [[32, 32], [64, 64], [128, 128]]
    model.add_module("init_conv", ConvBNRelu(3, 32, kernel_size=7, stride=2, padding=3))

    in_channels = 32
    block_counter = 0
    for i, block in enumerate(config):
        for j in range(len(config[i])):
            out_channels = config[i][j]
            stride = 2 if i != 0 and j == 0 else 1
            # YOUR CODE: add ResNetBlock2 module to model
            block_name = f"res_block{block_counter}"
            model.add_module(
                block_name, ResNetBlock2(in_channels, out_channels, stride=stride)
            )

            in_channels = out_channels
            block_counter += 1

    model.add_module("pool", GlobalAveragePool((2, 3)))
    model.add_module("logits", nn.Linear(out_channels, 200))
    return model

Тогда давайте потренируем нашу сеть. Обычно после обучения в течение 30 эпох вы должны получить нейронную сеть, которая предсказывает метки с точностью >40% и дает около +1% профита по сравнению с vgg, из предыдущего эксперимента.

In [ ]:
# YOUR CODE: create resnet model, move it to 'device', create same optimizer as in previous experiment
model_r = create_resnet_like_network().to(device)
opt = torch.optim.Adam(model_r.parameters(), lr = 0.001)
train_loop(model_r, opt, train_batch_gen, val_batch_gen, num_epochs=5)

100%|██████████| 1563/1563 [03:54<00:00,  6.68it/s]


Epoch 1 of 5 took 236.244s
  training loss (in-iteration): 	4.698288
  validation accuracy: 			8.69 %


100%|██████████| 1563/1563 [03:55<00:00,  6.64it/s]


Epoch 2 of 5 took 237.432s
  training loss (in-iteration): 	3.923943
  validation accuracy: 			18.81 %


100%|██████████| 1563/1563 [04:03<00:00,  6.42it/s]


Epoch 3 of 5 took 245.421s
  training loss (in-iteration): 	3.501404
  validation accuracy: 			23.59 %


100%|██████████| 1563/1563 [04:02<00:00,  6.44it/s]


Epoch 4 of 5 took 244.546s
  training loss (in-iteration): 	3.233718
  validation accuracy: 			27.25 %


100%|██████████| 1563/1563 [04:07<00:00,  6.32it/s]


Epoch 5 of 5 took 249.400s
  training loss (in-iteration): 	3.035720
  validation accuracy: 			30.06 %


Если вы внимательно изучали нашу сеть resnet, то могли заметить, что она имеет почти в 2 раза больше параметров и в 2 раза глубже, чем vgg. Давайте определим сеть, сопоставимую vgg, удвоив количество уровней conv.

Наш новый сайт VGG-как архитектура будет [Сопу(16), усл. (16), MaxPool] + [Сопу(32), П(32), П(32), П(32), MaxPool] + [Сопу(64) отн(64) отн(64) отн(64), MaxPool] + [Сопу(128), Сопу(128), Сопу(128), Сопу(128)] + [GlobalAveragePooling] + [ФК(200) + softmax]

In [ ]:
model_v = create_vgg_like_network(config=[[16,16], [32,32,32,32], [64, 64, 64, 64], [128, 128, 128, 128]])
model_v = model_v.to(device)
opt = torch.optim.Adam(model_v.parameters())
train_loop(model_v, opt, train_batch_gen, val_batch_gen, num_epochs=5)

100%|██████████| 1563/1563 [04:05<00:00,  6.37it/s]


Epoch 1 of 5 took 247.457s
  training loss (in-iteration): 	4.709282
  validation accuracy: 			5.52 %


100%|██████████| 1563/1563 [04:08<00:00,  6.30it/s]


Epoch 2 of 5 took 250.015s
  training loss (in-iteration): 	4.157697
  validation accuracy: 			9.18 %


100%|██████████| 1563/1563 [04:08<00:00,  6.28it/s]


Epoch 3 of 5 took 251.555s
  training loss (in-iteration): 	3.809160
  validation accuracy: 			14.90 %


100%|██████████| 1563/1563 [04:06<00:00,  6.34it/s]


Epoch 4 of 5 took 249.555s
  training loss (in-iteration): 	3.575403
  validation accuracy: 			19.68 %


100%|██████████| 1563/1563 [04:04<00:00,  6.39it/s]


Epoch 5 of 5 took 247.318s
  training loss (in-iteration): 	3.401144
  validation accuracy: 			20.86 %


Видите ли вы выгоду от residual связей?

Качество сети vgg в этом эксперименте может быть даже хуже, чем качество сети vgg в первом эксперименте. Это связано с проблемой затухания градиента, которая затрудняет обучение глубоких нейронных сетей без residual связей.